In [1]:
import pandas as pd
from gurobipy import *
import openpyxl
from pprint import pprint

In [2]:
# reading the inputs from Excel workbook
wb = openpyxl.load_workbook("Excel_to_Gurobi_for_Network_Models.xlsx")

The attached Excel file contains 6 sheets, each corresponding to a different Network Model template:

- Transportation Problem
- Transshipment Problem
- Assignment Problem
- Shortest Path Problem
- Maximum Flow Problem
- Traveling Salesperson Problem

#### <span style="background-color: #FFFF00">NOTICE: Input the data into the cells of the chosen sheet, replace the sheet name indicated below, then run all cells.</span> 

In [3]:
main = wb['Transportation Problem']

In [4]:
# read objective
obj = main.cell(column=2, row=1).value
if obj =='max':
    obj = -1
elif obj =='min':
    obj = 1
else:
    raise NameError('Objective cannot be recognized')

In [5]:
# get variable name

column_name = []
for c in range(2,10000):
    v = main.cell(column=c, row=7).value
    if v is None or v == 'FLOW IN = OUT\n(Y/N)':
        break
    elif type(v) != str:
        column_name.append(str(v))            
    else:
        column_name.append(v)
        
n_column = len(column_name)


row_name = []
for c in range(8,10000):
    v = main.cell(column=1, row=c).value
    if v is None or v == 'DEMAND':
        break
    elif type(v) != str:
        row_name.append(str(v))            
    else:
        row_name.append(v)
        
n_row = len(row_name)


n_variables = n_column * n_row


variable_name = []
for i in range(0,n_row):
    for j in range(0,n_column):
        v_name = row_name[i] + '_TO_' + column_name[j]
        variable_name.append(v_name)

In [6]:
## new
# get obj coefficient  

obj_coeff=[]

v = main.cell(column=2, row=4).value

if v == 'MFP':
    for i in range(n_variables):
        v = 0
        obj_coeff.append(v)
    obj_coeff[-1] = -1
    
else:
    for i in range(0,n_row):
        for j in range(0,n_column):
            v = main.cell(column=j+2, row=i+8).value
            if v is None:
                if obj == -1:
                    v = -99999999
                else:
                    v = 99999999
            obj_coeff.append(v)

In [7]:
#read variable properties     
        
variable_type = []
v = main.cell(column=2, row=2).value

for c in range(n_variables):
    if not v:
        t = 'C' 
    elif v == 'cont':
        t = 'C'
    elif v == 'int':
        t = 'I'
    elif v == 'bin':
        t = 'B'
    else:
        raise NameError('Variable type cannot be recognized')
    variable_type.append(t)


In [8]:
# get constraints

# get constraints of supply

column_supply = 0

for c in range(2,10000):
    v = main.cell(column=c, row=7).value
    if v == 'SUPPLY':
        column_supply = c
        break

if column_supply == 0:
    print('ERROR! Please insert a column named [SUPPLY] which indicate the limit of supply')

    
v_supply = []
for i in range(n_row):
    v = main.cell(column=column_supply, row=i+8).value
    v_supply.append(v)
    

values = []
for i in range(n_row):
    r_values = [0] * (n_variables + 2)
    for j in range(n_column):
        r_values[i*n_column + j] = 1
    r_values[n_variables] = '<='
    r_values[n_variables + 1] = v_supply[i]
    if v_supply[i] is not None:
        values.append(r_values)

constr_supply = values

In [9]:
# get constraints of demand

row_demand = 0

for i in range(7,10000):
    v = main.cell(column=1, row=i).value
    if v == 'DEMAND':
        row_demand = i
        break

if row_demand == 0:
    print('ERROR! Please insert a row named [DEMAND] which indicate the constraints of demand')
    
v_demand = []
for i in range(n_column):
    v = main.cell(column=i+2, row=row_demand).value
    v_demand.append(v)

values = []

for i in range(n_column):
    r_values = [0] * (n_variables + 2)
    for j in range(n_row):
        r_values[i+ j*n_column] = 1
    r_values[n_variables] = '>='
    r_values[n_variables + 1] = v_demand[i]
    if v_demand[i] is not None:
        values.append(r_values)

constr_demand = values

In [10]:
# get constraints of non-neg

values = []

v = main.cell(column=2, row=3).value

if v == 'Y':
    values = []
    for i in range(n_variables):
        r_values = [0] * (n_variables + 2)
        r_values[i] = 1
        r_values[n_variables] = '>='
        r_values[n_variables + 1] = 0
        values.append(r_values)

constr_non_neg = values

In [11]:
# get constraints of the capacity of the node

values = []

for i in range(0,n_row):
    for j in range(0,n_column):
        v = main.cell(column=j+2, row=i+8).value
        if v is None:
            v = 0            
        values.append(v)

node_capacity = values


values = []

for i in range(0,n_variables):
    r_values = [0] * (n_variables + 2)
    r_values[i] = 1
    r_values[n_variables] = '<='
    r_values[n_variables + 1] = node_capacity[i]
    values.append(r_values)

constr_capacity = values

In [12]:
# get constraints of in_n_out

# get the column location of the field of transfer node('FLOW IN = OUT')

column_transfer_node = 0

for i in range(1,10000):
    v = main.cell(column=i, row=7).value
    if v == 'FLOW IN = OUT\n(Y/N)':
        column_transfer_node = i
        break

if column_transfer_node == 0:
    print('ERROR! The column [FLOW IN = OUT] is missing!')


# get the row name of transfer node

v_transfer_node_row = []

for i in range(len(v_supply)):
    v = main.cell(column=column_transfer_node, row=i+8).value
    if v == 'Y' or v == 'y':
        temp_row_name = main.cell(column=1, row=i+8).value
        temp = [str(temp_row_name),i]
        v_transfer_node_row.append(temp)
        
n_transfer_node_row = len(v_transfer_node_row)


# get the coordinate of transfer node

v_transfer_node = []

for i in range(len(v_transfer_node_row)):
    a = v_transfer_node_row[i][0]
    b = v_transfer_node_row[i][1]
    e = len(v_transfer_node)
    for j in range(len(column_name)):
        c = column_name[j]
        if a == c:
            temp = [a,[j,b]]
            v_transfer_node.append(temp)
            break
        
        
# constraints of in_n_out

values = []

n_v_transfer_node = len(v_transfer_node)

for i in range(n_v_transfer_node):
    r_values = [0] * (n_variables + 2)
    transfer_node_column = v_transfer_node[i][1][0]
    transfer_node_row = v_transfer_node[i][1][1]
    
    for j in range(n_row):
        r_values[j*n_column + transfer_node_column] = 1
        r_values[transfer_node_row*n_column + j] = -1
    r_values[n_variables] = '='
    r_values[n_variables + 1] = 0
    values.append(r_values)
    
constr_in_n_out = values

In [13]:
## read constraints as dataframe

v = main.cell(column=2, row=4).value

if v == 'MFP':
    cons = constr_in_n_out + constr_capacity
else:
    cons = constr_in_n_out + constr_supply  + constr_demand


v = main.cell(column=2, row=3).value

if v == 'Y':
    cons+= constr_non_neg
    
cons_df = pd.DataFrame()

cons_df = pd.DataFrame(cons)

cons_df.rename(columns={n_variables:'constraint type', 
                        n_variables+1:'RHS values'}, inplace=True)

cons_df

,0,1,2,3,4,5,6,7,8,constraint type,RHS values
0,1,1,1,0,0,0,0,0,0,<=,100
1,0,0,0,1,1,1,0,0,0,<=,300
2,0,0,0,0,0,0,1,1,1,<=,300
3,1,0,0,1,0,0,1,0,0,>=,300
4,0,1,0,0,1,0,0,1,0,>=,200
5,0,0,1,0,0,1,0,0,1,>=,200
6,1,0,0,0,0,0,0,0,0,>=,0
7,0,1,0,0,0,0,0,0,0,>=,0
8,0,0,1,0,0,0,0,0,0,>=,0
9,0,0,0,1,0,0,0,0,0,>=,0


In [14]:
# setup model
m = Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-15


In [15]:
# add variables
x=m.addVars(n_variables)
# set types, lb, ub of variables
for i in range(n_variables):
    x[i].setAttr('VarNAME', variable_name[i])
    x[i].setAttr('vType', variable_type[i])
#    x[i].setAttr('lb', variable_lb[i])
#    x[i].setAttr('ub', variable_ub[i])

In [16]:
# set objective
objective = quicksum(obj_coeff[i] * x[i] for i in range(n_variables))
m.setObjective(objective, obj)

In [17]:
# add constraints
if cons_df.shape[0]:
    ## add le constraints
    le_const = cons_df[cons_df['constraint type'] == '<=']
    le_const.reset_index(inplace=True, drop=True)
    for i in range(le_const.shape[0]):
        m.addConstr(quicksum(le_const.iloc[i,j]* x[j] for j in range(n_variables)) <= le_const['RHS values'][i])
    ## add ge constraints
    ge_const = cons_df[cons_df['constraint type'] == '>=']
    ge_const.reset_index(inplace=True, drop=True)
    for i in range(ge_const.shape[0]):
        m.addConstr(quicksum(ge_const.iloc[i,j]* x[j] for j in range(n_variables)) >= ge_const['RHS values'][i])
    ## add eq constraints
    eq_const = cons_df[cons_df['constraint type'] == '=']
    eq_const.reset_index(inplace=True, drop=True)
    for i in range(eq_const.shape[0]):
        m.addConstr(quicksum(eq_const.iloc[i,j]* x[j] for j in range(n_variables)) == eq_const['RHS values'][i])

In [18]:
## run the model
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 9 columns and 27 nonzeros
Model fingerprint: 0xe0492841
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+02]
Presolve removed 9 rows and 0 columns
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.750000e+01   0.000000e+00      0s
       9    3.0000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.000000000e+03


In [19]:
# print optimal objective value
m.objVal

3000.0

In [20]:
# print optimal decisions (if not listed, that means their optimal value = 0)
m.printAttr('X')


    Variable            X 
-------------------------
Winnipeg_TO_Edmonton          100 
Montreal_TO_Edmonton          200 
Montreal_TO_Toronto          100 
Halifax_TO_Toronto          100 
Halifax_TO_Ottawa          200 


# Branch: solution of Traveling Salesperson Problem(TSP)

In [21]:
# check the variable availability

v = main.cell(column=2, row=4).value

if v == 'TSP':
    
    for i in range(n_column):
        if column_name[i] not in row_name:
            print('ERROR! Some of the row name are not as same as the column name!')
            break

In [22]:
# iterate the result in TSP 

v = main.cell(column=2, row=4).value

if v == 'TSP':
    
    # transfer the result to list

    result_list = m.getVars()
    result_list_1 = [] 

    for i in range(len(result_list)):
        if (result_list[i].x > 0.99):
            temp = [i, result_list[i].varName]
            result_list_1.append(temp)

    for i in range(len(result_list_1)):
        s = result_list_1[i][1].split('_TO_',1)
        result_list_1[i][1] = s


    # count the numbers of the nodes of subtour

    n_result = len(result_list_1)

    subtour = []
    subtour.append(result_list_1[0])

    while subtour[-1][1][1] != subtour[0][1][0]:
        for i in range(n_result):
            if result_list_1[i][1][0] == subtour[-1][1][1]:
                subtour.append(result_list_1[i])

    n_subtour = len(subtour)

    subtour_index = []

    for i in subtour:
        subtour_index.append(i[0])


    # iteration the result

    while n_subtour != n_result:

        # add new constrain
        values = []
        constr_in_n_out_update = []

        n_v_transfer_node = len(v_transfer_node)

        for i in range(n_v_transfer_node):
            values = [0] * (n_variables + 2)

        for i in subtour_index:
            values[i] = 1

        values[n_variables] = '<='
        values[n_variables + 1] = n_subtour - 1

        constr_in_n_out_update = values

        cons_df.loc[len(cons_df.index)] = constr_in_n_out_update

        print(cons_df)

        m = Model()

        # add variables
        x=m.addVars(n_variables)
        # set types, lb, ub of variables
        for i in range(n_variables):
            x[i].setAttr('VarNAME', variable_name[i])
            x[i].setAttr('vType', variable_type[i])

        # set objective
        objective = quicksum(obj_coeff[i] * x[i] for i in range(n_variables))
        m.setObjective(objective, obj)

        # new constraints
        if cons_df.shape[0]:
            ## add le constraints
            le_const = cons_df[cons_df['constraint type'] == '<=']
            le_const.reset_index(inplace=True, drop=True)
            for i in range(le_const.shape[0]):
                m.addConstr(quicksum(le_const.iloc[i,j]* x[j] for j in range(n_variables)) <= le_const['RHS values'][i])
            ## add ge constraints
            ge_const = cons_df[cons_df['constraint type'] == '>=']
            ge_const.reset_index(inplace=True, drop=True)
            for i in range(ge_const.shape[0]):
                m.addConstr(quicksum(ge_const.iloc[i,j]* x[j] for j in range(n_variables)) >= ge_const['RHS values'][i])
            ## add eq constraints
            eq_const = cons_df[cons_df['constraint type'] == '=']
            eq_const.reset_index(inplace=True, drop=True)
            for i in range(eq_const.shape[0]):
                m.addConstr(quicksum(eq_const.iloc[i,j]* x[j] for j in range(n_variables)) == eq_const['RHS values'][i])

        ## run the model
        m.optimize()

        # print optimal decisions (if not listed, that means their optimal value = 0)
        m.printAttr('X')


        # transfer the result to list
        result_list = m.getVars()
        result_list_1 = [] 

        for i in range(len(result_list)):
            if (result_list[i].x > 0.99):
                temp = [i, result_list[i].varName]
                result_list_1.append(temp)

        for i in range(len(result_list_1)):
            s = result_list_1[i][1].split('_TO_',1)
            result_list_1[i][1] = s

        # count the numbers of the nodes of subtour
        n_result = len(result_list_1)

        subtour = []
        subtour.append(result_list_1[0])

        while subtour[-1][1][1] != subtour[0][1][0]:
            for i in range(n_result):
                if result_list_1[i][1][0] == subtour[-1][1][1]:
                    subtour.append(result_list_1[i])

        n_subtour = len(subtour)

        subtour_index = []

        for i in subtour:
            subtour_index.append(i[0])

        subtour_index

# Final Result

In [23]:
# print optimal objective value
m.objVal

3000.0

In [24]:
# print optimal decisions (if not listed, that means their optimal value = 0)
m.printAttr('X')


    Variable            X 
-------------------------
Winnipeg_TO_Edmonton          100 
Montreal_TO_Edmonton          200 
Montreal_TO_Toronto          100 
Halifax_TO_Toronto          100 
Halifax_TO_Ottawa          200 
